In [1]:
from langchain_community.document_loaders import PyPDFLoader

path_file = "/workspaces/Summarize_Documents/data/Aula 04 - Segurança Internacional_24-05-24.pdf"

loader = PyPDFLoader(file_path=path_file)

pages = loader.load_and_split()

# Combine the pages, and replace the tabs with spaces
text = ' '.join([page.page_content.replace('\t', ' ') for page in pages[2:]])



/home/vscode/.local/lib/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
print(text)

Estudos avançados  ................................ ................................ ................................ ................................ ................................ ...... 41 
Material de estudo complementar  ................................ ................................ ................................ ................................ ............  43 
 Licensed to Georgia Jacob Brolio - georgiabrolio@icloud.com 3     
Aula 4 – Seg ura nça internacio nal  
 
 
 
 
Itens do edital: 13. O Brasil e a agenda internacional: 13.10. Desarmamento e não proliferação; 13.11. 
Crimes de guerra e crimes contra a humanidade: genocídio, holocausto e o Tribunal Penal Internacional; 13.12. 
Terrorismo; 13.13. Narcotráfico, crime transnacional e crimes cibernéticos de alcance global; 16. A dimensão 
da segurança na política exterior do Brasil.  
 
Esta apostila foi preparada exclusivamente para os alunos do curso Política Internacional para 
Futuros Diplomatas  e pertence unicame

In [5]:
import re

def clean_text(text):
    # Remove extra spaces
    cleaned_text = re.sub(r' +', ' ', text)
    cleaned_text = cleaned_text.replace('\n', ' ')
    # Remove spaces around hyphens
    cleaned_text = re.sub(r'\s*-\s*', '', cleaned_text)
    # Remove 'Licensed to Georgia Jacob Brolio - georgiabrolio@icloud.com' followed by a number
    cleaned_text = re.sub(r'Licensed to Georgia Jacob Broliogeorgiabrolio@icloud\.com \d*', '', cleaned_text)
    # Remove any remaining specific pattern without number
    cleaned_text = re.sub(r'Licensed to Georgia Jacob Broliogeorgiabrolio@icloud\.com', '', cleaned_text)
    # Remove the especific pattern
    cleaned_text = re.sub(r'Esta apostila foi preparada exclusivamente para os alunos do curso Política Internacional para Futuros Diplomatas e pertence unicamente à pessoa cujos dados estão marcados acima. Seu compartilhamento, comercialização ou reprodução, com ou sem fins lucrativos, são proibidos. Em caso de dúvidas, escreva para contato@cursocacd.com\.', '', cleaned_text, flags=re.DOTALL)

    # Remove any extra spaces again
    cleaned_text = re.sub(r' +', ' ', cleaned_text).strip()


    return cleaned_text

clean_text = clean_text(text)

In [7]:
from config import set_environment 
set_environment()

from langchain_openai import OpenAI
llm = OpenAI()
Tokens = llm.get_num_tokens(clean_text)
print (f"Nós temos  {Tokens} tokens na apostila")


Nós temos  33957 tokens na apostila


In [8]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
text_splitter = SemanticChunker(
   OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)
docs = text_splitter.create_documents([clean_text])

In [9]:
import numpy as np
import openai
def get_embeddings(text):
   response = openai.embeddings.create(
       model="text-embedding-3-small",
       input=text
   )
   return response.data
embeddings=get_embeddings([doc.page_content for doc in docs]
)

In [10]:
import pandas as pd
content_list = [doc.page_content for doc in docs]
df = pd.DataFrame(content_list, columns=['page_content'])
vectors = [embedding.embedding for embedding in embeddings]
array = np.array(vectors)
embeddings_series = pd.Series(list(array))
df['embeddings'] = embeddings_series

In [25]:
import numpy as np
import faiss
from langchain_openai import ChatOpenAI

# Supondo que 'array' já esteja definido e carregado com os dados

# Convertendo para float32 se ainda não estiver
array = array.astype('float32')

num_points = array.shape[0]


# Definindo o número de clusters
num_clusters = min(50, num_points)

# Dimensionalidade dos vetores
dimension = array.shape[1]

# Treinando o KMeans com Faiss
kmeans = faiss.Kmeans(dimension, num_clusters, niter=20, verbose=True)
kmeans.train(array)

# Acessando diretamente os centróides
centroids = kmeans.centroids

# Criando um novo índice para o dataset original
index = faiss.IndexFlatL2(dimension)

# Adicionando o dataset original ao índice
index.add(array)

D, I = index.search(centroids, 1)

sorted_array = np.sort(I, axis=0)
sorted_array=sorted_array.flatten()
extracted_docs = [docs[i] for i in sorted_array]

model = ChatOpenAI(temperature=0,model="gpt-4o-mini")

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
                Você é uma aluna de relações internacionais e está estudando para uma prova. Você receberá os textos para resumir de forma natural e coerente.
                                          Busque destacar os aspectos mais importantes do texto.
                                          Evite fazer fazer menções do tipo "o texto diz que" ou "o autor afirma que". 
                                          Caso você não tenha recebido o texto para fazer o resumo, não escreva nada, siga adiante. Seja direto e objetivo.
                TEXTO:
                ```{text}```
                RESUMO:
                """
                )

chain= prompt| model

from tqdm import tqdm
final_summary = ""

for doc in tqdm(extracted_docs, desc="Processing documents"):
   # Get the new summary.
   new_summary = chain.invoke({"text": doc.page_content})
   # Update the list of the last two summaries: remove the first one and add the new one at the end.
   final_summary+=new_summary.content

print(final_summary)

WARNING clustering 44 points to 44 centroids: please provide at least 1716 training points


Number of training points (44) same as number of clusters, just copying


Processing documents: 100%|██████████| 44/44 [02:20<00:00,  3.18s/it]

Não recebi um texto para resumir.Não recebi nenhum texto para resumir.Não há texto disponível para resumo.Não recebi nenhum texto para resumir.O Brasil tem um papel importante na agenda internacional, abordando temas como desarmamento e não proliferação de armas, além de questões relacionadas a crimes de guerra e crimes contra a humanidade, incluindo genocídio e holocausto, com a atuação do Tribunal Penal Internacional. A luta contra o terrorismo também é um aspecto relevante na política externa brasileira.O texto aborda temas relevantes como narcotráfico, crime transnacional e crimes cibernéticos, destacando sua natureza global e o impacto na segurança. Além disso, menciona a importância da segurança na política exterior do Brasil, sugerindo que esses problemas transnacionais influenciam as estratégias diplomáticas do país.As armas de destruição em massa (ADM) englobam armas biológicas, nucleares, químicas e radiológicas. Em 1977, a Assembleia Geral das Nações Unidas definiu ADM como 